<center>
<h2>Online learning platform database - Cassandra</h2>
</center>

<h3>Preliminary operations: import csv files into Cassandra (<code>COPY FROM</code>)</h3>

A csv file can be imported into a Cassandra table via the <code>COPY FROM</code> CQL <a href = 'https://cassandra.apache.org/doc/4.1/cassandra/tools/cqlsh.html#copy-from'>command</a>, although for large files it is suggested, to rely on <a href = 'https://cassandra.apache.org/doc/latest/cassandra/operating/bulk_loading.html'><i>bulk</i> loading</a>, an operation that can be performed either with the <code>sstableloader</code> command or with the <code>nodetool import</code> command. Neither of the two is a DML command and they need being executed from the Cassandra shell.<br>A major limitation of these tools resides in the fact that they can import external files in the form of <i>sstables</i>, not directly csv files. Hence, <i>bulk-loading</i> external data that is not in sstable form, should first be stored into sstables, a process that requires using the <code>CQLSSTableWriter</code> Java class. After experimenting with a file with a couple of thousands of rows, I have resolved to rely on the <code>COPY FROM</code> CQL command.
<br>
<h4>
Syntax
</h4><br>
To use the <code>COPY FROM</code> command we must first select the keyspace (<code>USE <i>mykeyspace</i></code>) of the table that we want the csv data to be imported into. The table must exist within the selected keyspace and its schema must replicate the header of the csv file and related data types.
<br>
<code>
    COPY <i>mytable</i> FROM '<i>mycsvfile.csv</i>';
</code>
<br>
Equivalently, without the need to first select the keyspace, we could specify the table as a keyspace attribute: <br>
<code>
    COPY <i>mykeyspace.mytable</i> FROM '<i>mycsvfile.csv</i>';
</code>
<br>
It is to be considered that when we perform a <code>CREATE TABLE</code> CQL command there is no guarantee that the output table will keep the inputted column order. So, although in table creation we might have specified the same column order of the first row of the csv (header), when importing it is advisable to specify the column list (within round brackets after the table name) in the same order as it appears in the csv header.
<br>
<code>
    COPY <i>tablename(column1, column2, ...)</i> FROM '<i>mycsvfile.csv</i>';
</code>
<h4>
Options
</h4><br>
Options are specified after the keyword <code>WITH</code> and separated by the keyword <code>AND</code>. Options are useful to change the CSV format. In fact, by default, Cassandra expects the CSV data to consist of fields separated by commas (,), records separated by line separators (a newline, \r\n), field values enclosed in double-quotation marks ("") and a header row not to be present.
<br>
<br>
- <code><b>HEADER</b></code><br>
If the first row of the csv file contains the column names, we must specify <code>HEADER = true</code> after the filename, so that it is ignored when importing.
<br>
<code>
    COPY <i>tablename(column1, column2, ...)</i> FROM '<i>mycsvfile.csv</i>' 
        WITH HEADER = true;
</code>
<br>
- <code><b>DELIMITER</b></code><br>
To avoid problems in column identification, it is also advisable to specify the character used within the csv file to separate columns.
<br>
<code>
    COPY <i>tablename(column1, column2, ...)</i> FROM '<i>mycsvfile.csv</i>' 
        WITH HEADER = true AND DELIMITER = ',';
</code>
<br>
- <code><b>QUOTE</b></code><br>
If there are commas within column values, column values usually come written within double quotes. If a different enclosing characyer is used, it must be specified with the <code>QUOTE</code> option.
<br>
<code>
    COPY <i>tablename(column1, column2, ...)</i> FROM '<i>mycsvfile.csv</i>' 
        WITH HEADER = true AND DELIMITER = ',' AND QUOTE = '"';
</code>
<br>
However, in my datasets, usually I don't use double quotation to enclose field values, so to preserve data types. Hence, only strings that contain commas are enclosed in double quotes. This may cause <code>COPY FROM</code> to experience troubles because, if <code>QUOTE</code> <b>is used</b>, not every field value comes with this kind of enclosure, while, if <code>QUOTE</code> <b>is not used</b>,<code>COPY FROM</code> will likely fail in identifying the correct number of field when reaching a row where some values are double quoted. This is likely to happen for the <i>courseName</i> field: course names may include commas. To increase compatibility, it is then advisable to double quote dictionary values when creating the data with Python and leave to the DBMSs the task of identifying the correct data type.
<br>
<h4>
Notes when running Cassandra from a Docker container
</h4><br>
In loading the csv file from the local machine, we must take into account the usual feature of Docker virual environments. They come with a file system of their own, so a DBMS run from within a container has access to this file system, not to the local machine file system. Hence, the csv file must be imported into the container via the usual <code>docker cp</code> command.
<br>
<code>
    docker cp path/mycsvfile container:/path
</code>

Contrary to what has been specified in the MySQL and MongoDB cases, I run the <code>COPY FROM</code> command from <i>cqlsh</i>, not from a system folder or from a container shell. Also, the file has been copied in the root folder of the container file system, hence I don't need to include a path for the csv file, I just specify its name within single quotes.

<h3>Python - Cassandra interaction</h3>

Interaction between a Python API and a Cassandra DBMS requires the installation of a specific driver. The usual list of drivers for various programming languages is provided in the <a href = 'https://www.mongodb.com/docs/drivers/'>Client drivers</a> web page of the Cassandra website: <a href = 'https://docs.datastax.com/en/developer/python-driver/3.28/'>Datastax Python driver</a> is the one suggested for a Python programming environment.<br>After having installed the driver, it can be imported into a Python environment the usual way. In particular, we need to setup an instance of Cluster for the Cassandra cluster we want to interact with, so we first import the <code>Cluster</code> class from the driver.

In [124]:
from cassandra.cluster import Cluster

<h4>
Establishing a connection to a Cassandra cluster
</h4><br>
When instantiating a cluster, the default behaviour of the driver is to try to connect to a Cassandra instance on the local machine. The optional local machine's name (<i>localhost</i>) or ip address (<i>127.0.0.1</i>) can be specified as an argument in the form of a string element of a Python list. Alternatively, the ip addresses of the nodes in the cluster can be provided as string elements of a Python list.

In [125]:
# Equivalent syntaxes
#cluster = Cluster()
#cluster = Cluster(['localhost'])
cluster = Cluster(['127.0.0.1'])

Cluster instantiation is not enough to establish a connection to the nodes: establishing a connection requires the creation of a <code>session</code> class via the <code>connect()</code> method of the cluster.

In [126]:
session = cluster.connect()

<h4>
Choosing a database
</h4><br>
A list of available databases (Cassandra keyspaces) is stored as a dictionary within the <code>cluster.metadata.keyspaces</code> object. The keyspaces' names are the dictionary keys, while the values represent keyspaces' memory locations. Iterating over the dictionary keys lets us easily print out the desired information.

In [158]:
print('List of keyspaces in the Cassandra instance:\n')
for keyspace in cluster.metadata.keyspaces.keys():
    print(keyspace)

List of keyspaces in the Cassandra instance:

system_auth
hotel
system_schema
dbb_cassandra_test
system_distributed
system
system_traces
system_views
system_virtual_schema


A session can take a keyspace name as argument, in that case that is the default keyspace for queries executed through that session. A keyspace can also be assigned to the session via the <code>set_keyspace()</code> method or passing the CQL <code>USE <i>mykeyspace</i></code> command to the <code>execute()</code> method of the session.

In [128]:
#session.execute('USE dbb_cassandra_test')
session.set_keyspace('dbb_cassandra_test')

<h4>
Executing a query
</h4><br>
Queries can be performed by passing the query string to the <code>execute()</code> method of the session (we must have previously chosen the keyspace).

In [165]:
res = session.execute('SELECT studentID, firstName, lastName FROM datasetsmall')

<h4>
Displaying a query result
</h4><br>
The query result is a set of rows in the form of named tuples. Each element has a name corresponding to its projected column name and a value. We can access the values by position, so we can display them by attaching an index to the elements (rows) of the result set we are looping over.

In [166]:
print(' ' * (10 - len('student ID')), 'student ID', ' ' * (25 - len('name')), 'name', ' ' * (25 - len('surname')), 'surname')
for row in res:
    print(' ' * (10 - len(str(row[0]))), row[0], ' ' * (25 - len(row[1])), row[1], ' ' * (25 - len(row[2])), row[2])

 student ID                       name                    surname
         40                      Jason                       Lind
         33                    Gabriel                    Ribeiro
         17                      Malin                    Evensen
         42                      Gavin                      Smith
         27                       Lina               van de Kooij
         27                       Lina               van de Kooij
          8                    Artūras                   Gronskis
         19                      Gatik                        Sen
         25                    Melissa                     Mendes
         24                      Oldağ                    Erdoğan
         39                    Kathrin                       Bien
         37                     Ingrid                   Karlsson
         19                      Gatik                        Sen
         42                      Gavin                      Smith
         4

After having accessed the query result, the memory used by the object result is freed, and the query result cannot be accessed any longer. The query must be performed again to access the result.

<h4>
Measuring and displaying the query execution time
</h4><br>
To display the query execution time we can use the Python <a href = 'https://docs.python.org/3/library/time.html'><i>time</i></a> module and its <code>time</code> function. The function returns the system time at a floating point precision, so the query execution time can be measured as a large number of fractions of a second. It is sufficient to assign the time before the query execution to a variable and the time after the query execution to another variable. The difference between the two variables will measure the query execution. Obviously, the time for the Python API to connect to the Cassandra cluster and the time to return to the Python API after the query execution will be summed up to the query execution time at the DBMS level.

In [183]:
import time
start = time.time()
session.execute('SELECT studentID, firstName, lastName FROM datasetsmall')
end = time.time()
print(end - start)

0.1501610279083252


In [181]:
import time
start1 = time.time()
session.execute('SELECT studentID, firstName, lastName FROM datasetsmall WHERE courseID = 192 ALLOW FILTERING')
end1 = time.time()
print(end1 - start1)

0.07848286628723145
